<a href="https://colab.research.google.com/github/shreyaprabhu19/MinorProject_DS23/blob/main/Classification_using_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

2023-04-22 17:16:07.939023: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
from astropy.io import ascii
import matplotlib.patches as mpatches
from matplotlib import rc,rcParams
from astropy.table import Table
import matplotlib
%matplotlib inline
%matplotlib notebook
from astropy.io import fits
from numpy import arange
import estimate_source_angles_detectors
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
def find_brightest_det(file_list, category, dataset):
    det = []
    for file in file_list:
        event_filename = '/home/shreya/FermiData/'+dataset+'/' + category +'/trigdat/'+ file
        file = fits.open(event_filename, memmap=True)
        ra_obj = file[0].header['RA_OBJ']
        dec_obj = file[0].header['DEC_OBJ']
        file.close()
        brightest_nai, bright_nais, brightest_bgo = estimate_source_angles_detectors.angle_to_grb(ra_obj,dec_obj,event_filename)
        det.append(brightest_nai)
    return det

In [ ]:
def time_data(file,det):
    data = file[2].data
    trig_time = file[0].header['TRIGTIME']
    pha = []
    for i in data:
        pha.append(i[0])
    arr = np.array(pha)
    time = arr-trig_time
    return time

In [ ]:
def read_data(det, category,lists, dataset):
    data = []
    trigger = []
    for i in range(len(det)):
        temp = lists[i].split("_")[3]
        trigger.append(temp)
        event_filename = '/home/shreya/FermiData/'+dataset+'/' + category +'/tte/glg_tte_'+ det[i] + '_'+ temp + '_v00.fit'
        tte = fits.open(event_filename, memmap=True)
        data.append(time_data(tte, det[i]))
    return data

In [ ]:
 def return_hist(data, bin_size, x_range, x_start): #x_range = 100
    histograms = []
    n_bins = []
    for i in range(len(data)):
        n_bin = math.ceil(data[i][len(data[i])-1]/bin_size)
        n_bins.append(n_bin)
        hist, bins = np.histogram(data[i], bins=n_bin)
        l = len(data[i])
        start = math.floor((-x_start-data[i][0])/bin_size)
        end = start + int(x_range/bin_size)
        histograms.append(hist[start:end])
    return histograms

In [ ]:
def bin_hist(data, bins):
#     x_range = [100, 20, 10, 2, 0.2] #x2
#     x_start = [10, 10, 5, 1, 0.1]
    x_range = [110, 110, 110, 110, 110] #x2
    x_start = [10, 10, 10, 10, 10]
    hist_temp = []
    for i in range(len(bins)):
        hist_temp.append(return_hist(data, bins[i], x_range[i], x_start[i]))
    return hist_temp

In [ ]:
#reading testing and training data files
file_list1 = os.listdir('/home/shreya/FermiData/training_data2/GRB_1/trigdat')
file_list2 = os.listdir('/home/shreya/FermiData/training_data2/TGF_1/trigdat')
file_list = file_list1 + file_list2
det1 = find_brightest_det(file_list1, "GRB_1", "training_data2")
det2 = find_brightest_det(file_list2, "TGF_1", "training_data2")
det = det1 + det2
data1 = read_data(det1, "GRB_1", file_list1, "training_data2")
data2 = read_data(det2, "TGF_1", file_list2, "training_data2")
data = data1 + data2
test_list1 = os.listdir('/home/shreya/FermiData/testing_data2/grbs/trigdat')
test_list2 = os.listdir('/home/shreya/FermiData/testing_data2/tgfs/trigdat')
test_list = test_list1 + test_list2
test_det1 = find_brightest_det(test_list1, "grbs", "testing_data2")
test_det2 = find_brightest_det(test_list2, "tgfs", "testing_data2")
test_det = test_det1 + test_det2
test_data1 = read_data(test_det1, "grbs", test_list1, "testing_data2")
test_data2 = read_data(test_det2, "tgfs", test_list2, "testing_data2")
test_data = test_data1 + test_data2

In [ ]:
#Choosen bin sizes according to the X-ray transients considered
bins = [1, 0.1, 0.05, 0.01, 0.001]
histograms = []
hist_temp2 = bin_hist(test_data, bins)
bins = [1, 0.1, 0.05, 0.01, 0.001]
histograms = []
hist_temp = bin_hist(data, bins)

In [ ]:
hist_2d = []
for j in range(len(data)):
    final = np.zeros((5,110000))
    for i in range(len(bins)):
        for k in range(len(hist_temp[i][j])):
            final[i][k] = hist_temp[i][j][k]
    hist_2d.append(final)

In [ ]:
hist_2d_test = []
for j in range(len(test_data)):
    final = np.zeros((5, 110000))
    for i in range(len(bins)):
        for k in range(len(hist_temp2[i][j])):
            final[i][k] = hist_temp2[i][j][k]
    hist_2d_test.append(final)

In [ ]:
a1 = np.empty(33)
a1.fill(0)
a2 = np.empty(30)
a2.fill(1)
label = np.append(a1, a2)
y_train = np.array(label)
np.shape(y_train)

(63,)

In [ ]:
a1 = np.empty(13)
a1.fill(0)
a2 = np.empty(12)
a2.fill(1)
y_test = np.append(a1, a2)
y_test = np.array(y_test)
np.shape(y_test)

(25,)

The classification algorithm using a single layered convolutional neural network is run for various values of parameters
.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

# Set up the data
train_data = X_train
train_labels = y_train

# Define the model
model = models.Sequential()
model.add(layers.Conv1D(256, 3, activation='relu', input_shape=(5, 110000)))
model.add(layers.MaxPooling1D(2))
model.add(layers.Flatten())
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_data, train_labels, epochs=20, batch_size=4)

# Test the model on some new data
test_data = X_test
predictions2 = model.predict(test_data)


Epoch 1/20
10/10 [==============================] - 3s 252ms/step - loss: 736.9598 - accuracy: 0.5000
Epoch 2/20
10/10 [==============================] - 3s 253ms/step - loss: 151.3690 - accuracy: 0.5500
Epoch 3/20
10/10 [==============================] - 3s 253ms/step - loss: 20.0561 - accuracy: 0.5750
Epoch 4/20
10/10 [==============================] - 3s 254ms/step - loss: 4.2977 - accuracy: 0.6750
Epoch 5/20
10/10 [==============================] - 3s 254ms/step - loss: 31.5687 - accuracy: 0.3000
Epoch 6/20
10/10 [==============================] - 3s 253ms/step - loss: 8.1809 - accuracy: 0.5500
Epoch 7/20
10/10 [==============================] - 3s 253ms/step - loss: 43.6080 - accuracy: 0.5000
Epoch 8/20
10/10 [==============================] - 3s 252ms/step - loss: 27.5199 - accuracy: 0.6250
Epoch 9/20
10/10 [==============================] - 3s 253ms/step - loss: 14.5107 - accuracy: 0.5250
Epoch 10/20
10/10 [==============================] - 3s 255ms/step - loss: 3.7306 - accurac

In [ ]:
binary_predictions = (predictions2 > 0.5).astype(int)
binary_predictions

array([[0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1]])

In [ ]:
acc = accuracy_score(y_test, binary_predictions)
print(f"Accuracy: {acc}")

Accuracy: 0.8


In [ ]:
#Reading another data set of testing and training data
test_list3 = os.listdir('/home/shreya/FermiData/test_new/grbs/trigdat')
test_list4 = os.listdir('/home/shreya/FermiData/test_new/tgfs/trigdat')
test_listII = test_list3 + test_list4
test_det3 =  find_brightest_det(test_list3, "grbs", "test_new")
test_det4 = find_brightest_det(test_list4, "tgfs", "test_new")
test_detII = test_det4 +test_det3
test_data3 = read_data(test_det3, "grbs", test_list3, "test_new")
test_data4 = read_data(test_det4, "tgfs", test_list4, "test_new")
test_dataII = test_data4 + test_data4

In [ ]:
bins = [1, 0.1, 0.05, 0.01, 0.001]
histograms = []
hist_temp2II = bin_hist(test_dataII, bins)

In [ ]:
hist_2d_testII = []
for j in range(len(test_dataII)):
    final = np.zeros((5, 110000))
    for i in range(len(bins)):
        for k in range(len(hist_temp2II[i][j])):
            final[i][k] = hist_temp2II[i][j][k]
    hist_2d_testII.append(final)

The classification algorithm using a single layered convolutional neural network is run for various values of parameters using the early stopping technique, few of the outputs can be seen below.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

# Set up the data
train_data = X_train
train_labels = y_train

# Define the model
model = models.Sequential()
model.add(layers.Conv1D(256, 3, activation='relu', input_shape=(5, 110000)))
model.add(layers.MaxPooling1D(2))
model.add(layers.Flatten())
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
x_val = X_test
y_val = y_test
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, mode='min')

# train the model with early stopping
history = model.fit(X_train, y_train, epochs=35, batch_size=4, validation_data=(x_val, y_val), callbacks=[early_stop])
# model.fit(train_data, train_labels, epochs=18, batch_size=4)

# Test the model on some new data
test_data = X_test
predictions4 = model.predict(test_data)

binary_predictions4 = (predictions4 > 0.5).astype(int)
binary_predictions4

Epoch 1/35
16/16 [==============================] - 5s 264ms/step - loss: 317.6365 - accuracy: 0.4444 - val_loss: 9.3449 - val_accuracy: 0.4800
Epoch 2/35
16/16 [==============================] - 4s 256ms/step - loss: 30.7420 - accuracy: 0.4762 - val_loss: 46.6217 - val_accuracy: 0.5200
Epoch 3/35
16/16 [==============================] - 4s 256ms/step - loss: 39.0533 - accuracy: 0.4444 - val_loss: 19.7238 - val_accuracy: 0.4800
Epoch 4/35
16/16 [==============================] - 4s 257ms/step - loss: 32.1314 - accuracy: 0.5079 - val_loss: 31.4195 - val_accuracy: 0.5200
Epoch 5/35
16/16 [==============================] - 4s 256ms/step - loss: 20.2229 - accuracy: 0.5397 - val_loss: 23.6634 - val_accuracy: 0.5200
Epoch 6/35
16/16 [==============================] - 4s 256ms/step - loss: 20.0273 - accuracy: 0.4127 - val_loss: 24.1147 - val_accuracy: 0.5200
Epoch 7/35
16/16 [==============================] - 4s 256ms/step - loss: 18.7546 - accuracy: 0.4762 - val_loss: 15.7737 - val_accuracy:

array([[0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1]])

In [ ]:
acc = accuracy_score(y_test, binary_predictions4)
print(f"Accuracy: {acc}")

Accuracy: 0.76


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models


# Set up the data
# train_data = X_train
# train_labels = y_train
train_data, x_val, train_labels, y_val = train_test_split(X_train, y_train)

# Define the model
model = models.Sequential()
model.add(layers.Conv1D(256, 3, activation='relu', input_shape=(5, 110000)))
model.add(layers.MaxPooling1D(2))
model.add(layers.Flatten())
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
# x_val = X_test
# y_val = y_test
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, mode='min')

# train the model with early stopping
history = model.fit(X_train, y_train, epochs=35, batch_size=4, validation_data=(x_val, y_val), callbacks=[early_stop])
# model.fit(train_data, train_labels, epochs=18, batch_size=4)

# Test the model on some new data
test_data = X_test
predictions4 = model.predict(test_data)

binary_predictions4 = (predictions4 > 0.5).astype(int)
binary_predictions4

Epoch 1/35
16/16 [==============================] - 5s 261ms/step - loss: 198.0213 - accuracy: 0.4762 - val_loss: 31.1639 - val_accuracy: 0.5625
Epoch 2/35
16/16 [==============================] - 4s 254ms/step - loss: 27.7698 - accuracy: 0.4286 - val_loss: 5.4507 - val_accuracy: 0.5625
Epoch 3/35
16/16 [==============================] - 4s 253ms/step - loss: 48.6603 - accuracy: 0.5079 - val_loss: 33.6812 - val_accuracy: 0.5000
Epoch 4/35
16/16 [==============================] - 4s 253ms/step - loss: 40.9856 - accuracy: 0.4444 - val_loss: 37.3411 - val_accuracy: 0.5000
Epoch 5/35
16/16 [==============================] - 4s 254ms/step - loss: 14.4252 - accuracy: 0.4921 - val_loss: 1.1163 - val_accuracy: 0.6250
Epoch 6/35
16/16 [==============================] - 4s 253ms/step - loss: 7.7231 - accuracy: 0.6032 - val_loss: 6.0750 - val_accuracy: 0.5000
Epoch 7/35
16/16 [==============================] - 4s 254ms/step - loss: 14.6343 - accuracy: 0.5873 - val_loss: 10.9024 - val_accuracy: 0.

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0]])

In [ ]:
acc = accuracy_score(y_test, binary_predictions4)
print(f"Accuracy: {acc}")

Accuracy: 0.68


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models


# Set up the data
# train_data = X_train
# train_labels = y_train
train_data, x_val, train_labels, y_val = train_test_split(X_train, y_train)

# Define the model
model = models.Sequential()
model.add(layers.Conv1D(256, 3, activation='relu', input_shape=(5, 110000)))
model.add(layers.MaxPooling1D(2))
model.add(layers.Flatten())
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
# x_val = X_test
# y_val = y_test
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, mode='min')

# train the model with early stopping
history = model.fit(X_train, y_train, epochs=35, batch_size=4, validation_data=(x_val, y_val), callbacks=[early_stop])
# model.fit(train_data, train_labels, epochs=18, batch_size=4)

# Test the model on some new data
test_data = X_test
predictions4 = model.predict(test_data)

# acc = accuracy_score(y_test, binary_predictions4)
# print(f"Accuracy: {acc}")

Epoch 1/35
16/16 [==============================] - 5s 263ms/step - loss: 568.3583 - accuracy: 0.4603 - val_loss: 48.1397 - val_accuracy: 0.4375
Epoch 2/35
16/16 [==============================] - 4s 254ms/step - loss: 19.7663 - accuracy: 0.5079 - val_loss: 26.0867 - val_accuracy: 0.5625
Epoch 3/35
16/16 [==============================] - 4s 255ms/step - loss: 53.7270 - accuracy: 0.5079 - val_loss: 34.7788 - val_accuracy: 0.5625
Epoch 4/35
16/16 [==============================] - 4s 255ms/step - loss: 12.8313 - accuracy: 0.4603 - val_loss: 17.2844 - val_accuracy: 0.6250
Epoch 5/35
16/16 [==============================] - 4s 254ms/step - loss: 24.6864 - accuracy: 0.3968 - val_loss: 13.1949 - val_accuracy: 0.4375
Epoch 6/35
16/16 [==============================] - 4s 254ms/step - loss: 27.7270 - accuracy: 0.4921 - val_loss: 30.5106 - val_accuracy: 0.4375
Epoch 7/35
16/16 [==============================] - 4s 255ms/step - loss: 15.3978 - accuracy: 0.4921 - val_loss: 5.4903 - val_accuracy:

In [ ]:
binary_predictions4 = (predictions4 > 0.5).astype(int)

acc = accuracy_score(y_test, binary_predictions4)
print(f"Accuracy: {acc}")

Accuracy: 0.36


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

# Set up the data
train_data = X_train
train_labels = y_train

# Define the model
model = models.Sequential()
model.add(layers.Conv1D(256, 3, activation='relu', input_shape=(5, 110000)))
model.add(layers.MaxPooling1D(2))
model.add(layers.Flatten())
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_data, train_labels, epochs=35, batch_size=4)

# Test the model on some new data
test_data = X_test
predictions2 = model.predict(test_data)
binary_predictions2 = (predictions2 > 0.5).astype(int)

acc = accuracy_score(y_test, binary_predictions2)
print(f"Accuracy: {acc}")

Epoch 1/35
16/16 [==============================] - 4s 251ms/step - loss: 541.5580 - accuracy: 0.5079
Epoch 2/35
16/16 [==============================] - 4s 251ms/step - loss: 23.5358 - accuracy: 0.5079
Epoch 3/35
16/16 [==============================] - 4s 251ms/step - loss: 23.4731 - accuracy: 0.5079
Epoch 4/35
16/16 [==============================] - 4s 251ms/step - loss: 62.5990 - accuracy: 0.4762
Epoch 5/35
16/16 [==============================] - 4s 251ms/step - loss: 27.9082 - accuracy: 0.4762
Epoch 6/35
16/16 [==============================] - 4s 251ms/step - loss: 16.8127 - accuracy: 0.5238
Epoch 7/35
16/16 [==============================] - 4s 251ms/step - loss: 31.7418 - accuracy: 0.5238
Epoch 8/35
16/16 [==============================] - 4s 251ms/step - loss: 11.4106 - accuracy: 0.5397
Epoch 9/35
16/16 [==============================] - 4s 250ms/step - loss: 3.9499 - accuracy: 0.6825
Epoch 10/35
16/16 [==============================] - 4s 250ms/step - loss: 5.4069 - accurac